In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [39]:
#Set up Intial array
X = 17.13
K_b = 0.75
Y = 0
B_w = 30
Lambda_deg = 32 #degrees
def get_K_s_val(X,Y,B_w,K_b,Lambda_deg):
    Lambda_rad = Lambda_deg*np.pi/180
    X_bar = X/(B_w*0.5)
    Y_bar = Y/(B_w*0.5)
    r_bar = np.sqrt(X_bar**2+Y_bar**2)
    s_bar = K_b*np.tan(Lambda_rad)
    t_bar = np.sqrt((X_bar-s_bar)**2+Y_bar**2+K_b**2)
    t_0bar = np.sqrt(X_bar**2+Y_bar**2+K_b**2)
    top = 1 + (X_bar-s_bar)/(t_bar) + (X_bar*(r_bar+t_bar)*(t_0bar**2 - X_bar**2))/(r_bar*t_bar*(r_bar*t_bar+r_bar**2 -X_bar*s_bar))
    bottom = 1 + (X_bar*(r_bar**2 + t_0bar**2 -X_bar**2))/(t_0bar*r_bar**2 )
    K_s = top/bottom
    return K_s
K_s = get_K_s_val(X,Y,B_w,K_b,Lambda_deg)
print(K_s)


1.0439788293848493


In [40]:
# CL solver

def get_CL_val(W,gamma,rho,vinf,sw):
    return W*np.cos(gamma*np.pi/180)/(0.5*rho*vinf**2 *sw)
CL = get_CL_val(20500,0,0.0023769,225,300)
print(CL)


1.1357602243957488


In [41]:
#Side wash estimator
#Equation for x_d
def x_d_equation(x_hat,gamma):
    return x_hat-np.tan(gamma*np.pi/180)
#Full equation estimator given known information
def epsilon_s(es_0,es_beta,beta):
    return es_0 + es_beta*beta*(np.pi/180)

def epsilon_beta(Cl_w,Ra_w,b_w,K_v,K_b,x,y,z,gamma):
    #solve for x_hat,y_hat,z_hat,x_d
    x_hat = (2*x)/(b_w*K_b)
    y_hat = (2*y)/(b_w*K_b)
    z_hat = (2*z)/(b_w*K_b)
    x_d = x_d_equation(x_hat,gamma)
    #lead coefficient
    lead_co = -(K_v*x_d*Cl_w)/(Ra_w*K_b*np.pi**2)
    var1 = ((-2*y_hat*(z_hat-1))/(y_hat**2 + (z_hat-1)**2)**2)*(1+((x_d)/(np.sqrt(x_d**2 + y_hat**2 + (z_hat-1)**2))))
    var2 = ((2*y_hat*(z_hat+1))/(y_hat**2 + (z_hat+1)**2)**2)*(1+((x_d)/(np.sqrt(x_d**2 + y_hat**2 + (z_hat+1)**2))))
    var3 = (y_hat/(y_hat**2 + (z_hat-1)**2))* ((x_d*(z_hat-1))/((x_d**2 + y_hat**2 + (z_hat-1)**2)**(3/2)))
    var4 = (y_hat/(y_hat**2 + (z_hat+1)**2))* ((x_d*(z_hat+1))/((x_d**2 + y_hat**2 + (z_hat+1)**2)**(3/2)))
    return lead_co*(var1+var2-var3+var4)

def epsilon_beta_simplified(CL_w,RA_w,K_v,K_beta,K_b):
    return -(CL_w*K_v*K_beta)/(K_b*RA_w)

#function to get the K_beta value using the equation where it lies on axis of sym ie Z=0
def get_K_beta_val_Z_zero(K_b,b_w,gamma,x,y):
    x_hat = (2*x)/(b_w*K_b)
    y_hat = (2*y)/(b_w*K_b)
    x_d = x_d_equation(x_hat,gamma)
    var1 = ((4*y_hat*x_d)/(np.pi**2 *(y_hat**2 + 1)**2)) *(1+((x_d)/(np.sqrt(x_d**2 + y_hat**2 +1 ))))
    var2 = ((2*y_hat)/(np.pi**2 *(y_hat**2 + 1)**2)) *((x_d**2)/((x_d**2 + y_hat**2 +1)**(3/2)))
    return var1 + var2

def get_CL_v_effective(RA_eff,K_L,C_L_alpha_tilda):
    return C_L_alpha_tilda/((1+C_L_alpha_tilda/(1*np.pi*RA_eff))*(1+K_L))

In [42]:
#Quiz 11 answers
print(get_CL_val(22000,0,0.000364,500*(5280/3600),1000))
print(get_K_beta_val_Z_zero(0.80,100,10,37,10))
print(epsilon_beta_simplified(0.30,10,0.98,get_K_beta_val_Z_zero(0.80,100,10,37,10),0.80))
print(get_CL_v_effective(8,0.012,2*np.pi))

0.22477522477522482
0.11884776964844729
-0.004367655534580438
4.9669449068613325
